In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
import numpy as np
import pandas as pd
import os
# os.chdir("..")
%cd ..
%load_ext autoreload
%autoreload 2

d:\Playground\tabular\pytorch-tabular


# Utility Functions

In [2]:
def make_mixed_classification(n_samples, n_features, n_categories):
    X,y = make_classification(n_samples=n_samples, n_features=n_features, random_state=42, n_informative=5)
    cat_cols = random.choices(list(range(X.shape[-1])),k=n_categories)
    num_cols = [i for i in range(X.shape[-1]) if i not in cat_cols]
    for col in cat_cols:
        X[:,col] = pd.qcut(X[:,col], q=4).codes.astype(int)
    col_names = [] 
    num_col_names=[]
    cat_col_names=[]
    for i in range(X.shape[-1]):
        if i in cat_cols:
            col_names.append(f"cat_col_{i}")
            cat_col_names.append(f"cat_col_{i}")
        if i in num_cols:
            col_names.append(f"num_col_{i}")
            num_col_names.append(f"num_col_{i}")
    X = pd.DataFrame(X, columns=col_names)
    y = pd.Series(y, name="target")
    data = X.join(y)
    return data, cat_col_names, num_col_names

def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} | {tag} F1: {val_f1}")

# Generate Synthetic Data 

First of all, let's create a synthetic data which is a mix of numerical and categorical features

In [3]:
data, cat_col_names, num_col_names = make_mixed_classification(n_samples=10000, n_features=20, n_categories=4)
train, test = train_test_split(data, random_state=42)
train, val = train_test_split(train, random_state=42)

# Importing the Library

In [4]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig

## Category Embedding Model

In [5]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-512-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3
)

experiment_config = ExperimentConfig(project_name="PyTorch Tabular Example", run_name="synthetic_classification_cat_embedding", exp_watch="gradients", log_target="wandb", log_logits=True)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    experiment_config=experiment_config
)

In [6]:
tabular_model.fit(train=train, validation=val)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: manujosephv (use `wandb login --relogin` to force relogin)


GPU available: True, used: False
TPU available: None, using: 0 TPU cores
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type             | Params
------------------------------------------------------------
0 | embedding_layers       | ModuleList       | 60    
1 | normalizing_batch_norm | BatchNorm1d      | 32    
2 | linear_layers          | Sequential       | 818 K 
3 | loss                   | CrossEntropyLoss | 0     
------------------------------------------------------------
818 K     Trainable params
0         Non-trainable params
818 K     Total params
Finding best initial lr:  95%|█████████▌| 95/100 [00:03<00:00, 29.31it/s]
LR finder stopped early due to diverging loss.
Learning rate set to 0.003311311214825908

  | Name                   | Type             | Params
------------------------------------------------------------
0 | embedding_layers       | ModuleList       | 60    

In [7]:
result = tabular_model.evaluate(test)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': tensor(0.8748, device='cuda:0'),
 'train_accuracy': tensor(0.6827, device='cuda:0'),
 'train_loss': tensor(0.5106, device='cuda:0'),
 'valid_accuracy': tensor(0.8489, device='cuda:0'),
 'valid_loss': tensor(0.4546, device='cuda:0')}
--------------------------------------------------------------------------------



## Node Model

In [16]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=False, # Runs the LRFinder to automatically derive a learning rate
    batch_size=64,
    accumulate_grad_batches=16,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()

model_config = NodeConfig(
    task="classification",
    learning_rate = 1e-3
)

experiment_config = ExperimentConfig(project_name="PyTorch Tabular Example", 
                                     run_name="synthetic_classification_node", 
                                     exp_watch="gradients", 
                                     log_target="wandb", 
                                     log_logits=True)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    experiment_config=experiment_config
)

In [17]:
tabular_model.fit(train=train, validation=val)

D:\miniconda3\envs\df_encoder\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


global_step,99
_step,20
_runtime,73
_timestamp,1609174731


global_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▇▇▇█
_timestamp,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▇▇▇█


wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


D:\miniconda3\envs\df_encoder\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Checkpoint directory saved_models exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: False
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
D:\miniconda3\envs\df_encoder\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | dense_bloc

...

wandb: WARNING Step must only increase in log calls.  Step 299 < 300; dropping {'valid_loss': 0.6144665479660034, 'valid_accuracy': 0.7573333382606506, 'epoch': 59}.
wandb: WARNING Step must only increase in log calls.  Step 299 < 300; dropping {'train_accuracy': 0.7048855423927307, 'epoch': 59}.


wandb: WARNING Step must only increase in log calls.  Step 349 < 350; dropping {'valid_loss': 0.5891686677932739, 'valid_accuracy': 0.762666642665863, 'epoch': 69}.
wandb: WARNING Step must only increase in log calls.  Step 349 < 350; dropping {'train_accuracy': 0.7137323617935181, 'epoch': 69}.


wandb: WARNING Step must only increase in log calls.  Step 399 < 400; dropping {'valid_loss': 0.5640312433242798, 'valid_accuracy': 0.7706666588783264, 'epoch': 79}.
wandb: WARNING Step must only increase in log calls.  Step 399 < 400; dropping {'train_accuracy': 0.7132482528686523, 'epoch': 79}.


wandb: WARNING Step must only increase in log calls.  Step 449 < 450; dropping {'valid_loss': 0.5404821634292603, 'valid_accuracy': 0.7795555591583252, 'epoch': 89}.
wandb: WARNING Step must only increase in log calls.  Step 449 < 450; dropping {'train_accuracy': 0.7123679518699646, 'epoch': 89}.


wandb: WARNING Step must only increase in log calls.  Step 499 < 500; dropping {'valid_loss': 0.5181346535682678, 'valid_accuracy': 0.7857778072357178, 'epoch': 99}.
wandb: WARNING Step must only increase in log calls.  Step 499 < 500; dropping {'train_accuracy': 0.7202025055885315, 'epoch': 99}.


In [18]:
result = tabular_model.evaluate(test)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': tensor(0.7960, device='cuda:0'),
 'train_accuracy': tensor(0.7202, device='cuda:0'),
 'train_loss': tensor(0.4127, device='cuda:0'),
 'valid_accuracy': tensor(0.7858, device='cuda:0'),
 'valid_loss': tensor(0.5181, device='cuda:0')}
--------------------------------------------------------------------------------



## Node Model with Category Embeddings

In [13]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=False, # Runs the LRFinder to automatically derive a learning rate
    batch_size=64,
#     accumulate_grad_batches=16,
    max_epochs=100,
    min_epochs=10,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()

model_config = NodeConfig(
    task="classification",
    learning_rate = 1e-3,
    embed_categorical=True
)

experiment_config = ExperimentConfig(project_name="PyTorch Tabular Example", 
                                     run_name="synthetic_classification_node_cat_embed", 
                                     exp_watch="gradients", 
                                     log_target="wandb", 
                                     log_logits=True)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    experiment_config=experiment_config
)

In [14]:
tabular_model.fit(train=train, validation=val)

global_step,20
_step,24
_runtime,19
_timestamp,1609176642
valid_loss,0.69254
valid_accuracy,0.496
epoch,4
train_accuracy,0.67188


global_step,▁▂▄▆███████
_step,▁▁▂▄▅▇▇▇▇██
_runtime,▁▂▄▆▇▇▇████
_timestamp,▁▂▄▆▇▇▇████
valid_loss,█▆▃▁▁
valid_accuracy,█▇▄▁▁
epoch,▁▃▅▆█
train_accuracy,▁▄▄▃█


wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


D:\miniconda3\envs\df_encoder\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Checkpoint directory saved_models exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: False
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
D:\miniconda3\envs\df_encoder\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | embedd

In [16]:
result = tabular_model.evaluate(test)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': tensor(0.8088, device='cuda:0'),
 'train_accuracy': tensor(0.6652, device='cuda:0'),
 'valid_accuracy': tensor(0.7991, device='cuda:0'),
 'valid_loss': tensor(0.5047, device='cuda:0')}
--------------------------------------------------------------------------------



## TabNet Model

In [5]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=2048,
#     accumulate_grad_batches=16,
    max_epochs=50,
    min_epochs=10,
    early_stopping=None,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()

model_config = TabNetModelConfig(
    task="classification",
    learning_rate = 1e-5,
n_d = 16, n_a=16, n_steps=4,
)

experiment_config = ExperimentConfig(project_name="PyTorch Tabular Example", 
                                     run_name="synthetic_classification_tabnet", 
                                     exp_watch="gradients", 
                                     log_target="wandb", 
                                     log_logits=True)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    experiment_config=experiment_config
)

In [6]:
tabular_model.fit(train=train, validation=val)

wandb: Currently logged in as: manujosephv (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


D:\miniconda3\envs\df_encoder\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Checkpoint directory saved_models exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: False
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
D:\miniconda3\envs\df_encoder\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | tabnet | TabNet           | 

Learning rate set to 0.3311311214825908
Learning rate set to 0.3311311214825908

  | Name   | Type             | Params
--------------------------------------------
0 | tabnet | TabNet           | 29.0 K
1 | loss   | CrossEntropyLoss | 0     

  | Name   | Type             | Params
--------------------------------------------
0 | tabnet | TabNet           | 29.0 K
1 | loss   | CrossEntropyLoss | 0     


wandb: WARNING Step must only increase in log calls.  Step 3 < 36; dropping {'valid_loss': 0.9500188827514648, 'valid_accuracy': 0.5155555605888367, 'epoch': 1}.
wandb: WARNING Step must only increase in log calls.  Step 3 < 36; dropping {'train_accuracy': 0.512161910533905, 'epoch': 1}.


wandb: WARNING Step must only increase in log calls.  Step 6 < 37; dropping {'valid_loss': 0.9315098524093628, 'valid_accuracy': 0.5244444608688354, 'epoch': 2}.
wandb: WARNING Step must only increase in log calls.  Step 6 < 37; dropping {'train_accuracy': 0.5186787247657776, 'epoch': 2}.


...

wandb: WARNING Step must only increase in log calls.  Step 39 < 48; dropping {'valid_loss': 0.9935970306396484, 'valid_accuracy': 0.4942222237586975, 'epoch': 13}.
wandb: WARNING Step must only increase in log calls.  Step 39 < 48; dropping {'train_accuracy': 0.5033615231513977, 'epoch': 13}.


wandb: WARNING Step must only increase in log calls.  Step 42 < 49; dropping {'valid_loss': 0.9886074066162109, 'valid_accuracy': 0.5057777762413025, 'epoch': 14}.
wandb: WARNING Step must only increase in log calls.  Step 42 < 49; dropping {'train_accuracy': 0.5298593044281006, 'epoch': 14}.


wandb: WARNING Step must only increase in log calls.  Step 45 < 50; dropping {'valid_loss': 0.9777677059173584, 'valid_accuracy': 0.5040000081062317, 'epoch': 15}.
wandb: WARNING Step must only increase in log calls.  Step 45 < 50; dropping {'train_accuracy': 0.5017452239990234, 'epoch': 15}.


wandb: WARNING Step must only increase in log calls.  Step 48 < 51; dropping {'valid_loss': 0.9904037714004517, 'valid_accuracy': 0.5040000081062317, 'epoch': 16}.
wandb: WARNING Step must only increase in log calls.  Step 48 < 51; dropping {'train_accuracy': 0.5059657096862793, 'epoch': 16}.


wandb: WARNING Step must only increase in log calls.  Step 51 < 52; dropping {'valid_loss': 0.9911414980888367, 'valid_accuracy': 0.5182222127914429, 'epoch': 17}.
wandb: WARNING Step must only increase in log calls.  Step 51 < 52; dropping {'train_accuracy': 0.5149288177490234, 'epoch': 17}.


In [7]:
result = tabular_model.evaluate(test)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': tensor(0.4896, device='cuda:0'),
 'train_accuracy': tensor(0.5057, device='cuda:0'),
 'valid_accuracy': tensor(0.5324, device='cuda:0'),
 'valid_loss': tensor(0.9689, device='cuda:0')}
--------------------------------------------------------------------------------

